In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Lab 08 - Virtual Screening Problem
In this lab, your task is to create a predictive Machine Learning model that will be deployed as a Virtual Screening (VS) tool. VS is a powerful computational technique used to evaluate the desired biological activity of thousands of drug molecules. By using this technology, we can significantly reduce the time and expenses involved in the drug development pipeline.

In this lab, you will take on the role of a **Machine Learning Engineer** at an innovative Pharmaceutical company. Your team has been working on identifying new treatments for Chagas disease, and your job is to train a machine learning model that can be used as a virtual screening tool to identify potential candidates for the treatment. After conducting some research, you've found a promising dataset of a detailed [_in vivo_](https://ridgewayresearch.co.uk/parasite-diagnostics-laboratory/in-vitro-in-vivo-assays/#:~:text=In%20vivo%20assays%20are%20used,Assess%20their%20safety) assay where thousands of molecules were evaluated to determine their biological activity in suppressing Trypanosoma Cruzi, the causal agent of Chagas disease. With your team, you've decided that this high-quality dataset can be used to train the machine learning model.

Your team of **Feature Engineers** has extracted different types of molecular features that describe these drug molecules and has generated the following dataset: `cruzi_comp_activity_molecular_features.csv`. However, they've also informed you that for some molecules, some molecular features, specifically "NumberOfAromaticRings" and "TPSA", couldn't be extracted. It's now up to you, the ML Engineer, to decide how to handle this issue.

The outcome of the in vivo assay is recorded in the "EC50" column of the dataset. According to the report, any molecule with an EC50 activity of less than or equal to 1 $\mu M$ is considered active. This means that for a molecule to be deemed active, its EC50 value must be no greater than 1 $\mu M$. These active molecules can then be prioritized as potential candidates for a new treatment against Chagas disease.

You will need to analyze the data, consider the problem at hand, and decide whether to train a regression or a classification model. You will also need to select a correct and relevant performance metric for your model(s).

Please note that you are free to use any of the algorithms that were covered in the class for both regression and classification tasks. However, you must not import any other classifiers, such as Random Forests, XgBoost, or any other algorithm not covered in the class.

The task for today is analyse the performance of the different methods when applied to this interesting real-life problem, and obtain the best predictive model. As a side task, you can compare your own implementations of the algorithms with the sklearn ones and see if they behave in the same way and if they don't then try to think what could be causing these differences. 

In [17]:
# import the data
mol_df = pd.read_csv('cruzi_comp_activity_molecular_features.csv')

In [18]:
# Verificar valores ausentes e estatísticas básicas
missing_values = mol_df.isnull().sum()
statistics = mol_df.describe()

# Tratar valores ausentes imputando com a média de cada coluna
mol_df['NumberOfAromaticRings'].fillna(mol_df['NumberOfAromaticRings'].mean(), inplace=True)
mol_df['TPSA'].fillna(mol_df['TPSA'].mean(), inplace=True)

# Criar uma variável alvo binária
mol_df['Active'] = mol_df['EC50'] <= 1

# Dividir as features e a variável alvo
X = mol_df.drop(columns=['EC50', 'Active'])
y = mol_df['Active']

# Dividir o dataset em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Treinar e avaliar a Regressão Logística do sklearn
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)
y_pred_log_reg = log_reg.predict(X_test_scaled)

# Implementação manual da Regressão Logística
class ManualLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.m, self.n = X.shape
        self.weights = np.zeros(self.n)
        self.bias = 0

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            dw = (1 / self.m) * np.dot(X.T, (y_predicted - y))
            db = (1 / self.m) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        y_predicted_class = [1 if i > 0.5 else 0 for i in y_predicted]
        return y_predicted_class

# Treinar e avaliar a implementação manual da Regressão Logística
manual_log_reg = ManualLogisticRegression(learning_rate=0.01, n_iterations=1000)
manual_log_reg.fit(X_train_scaled, y_train)
y_pred_manual_log_reg = manual_log_reg.predict(X_test_scaled)

# Função para avaliar os modelos
def evaluate_model(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred)
    }

# Avaliar os modelos
results_sklearn = evaluate_model(y_test, y_pred_log_reg)
results_manual = evaluate_model(y_test, y_pred_manual_log_reg)

# Exibir resultados
missing_values, statistics, results_sklearn, results_manual

(CID                           0
 NumberOfRings                 0
 NumberOfAromaticRings        10
 ALogP                         0
 MolecularWeight               0
 RotatableBondCount            0
 HydrogenBondAcceptorCount     0
 HydrogenBondDonorCount        0
 TPSA                         10
 XlogP                         0
 EC50                          0
 dtype: int64,
                 CID  NumberOfRings  NumberOfAromaticRings        ALogP  \
 count  3.854000e+03    3854.000000            3844.000000  3854.000000   
 mean   7.376335e+06       3.473534               2.702133     0.016681   
 std    7.612063e+06       1.087264               0.961927     0.260351   
 min    1.201000e+03       0.000000               0.000000     0.000000   
 25%    2.157768e+06       3.000000               2.000000     0.000000   
 50%    3.582156e+06       3.000000               3.000000     0.000000   
 75%    1.200559e+07       4.000000               3.000000     0.000000   
 max    5.664295e+07  

## Round 2
After conducting some initial experiments, your team is not entirely satisfied with the obtained results. Several factors could be affecting the performance at this stage. One possible reason could be that the dataset size is too small, with insufficient assessed compounds. Another possible cause could be that the features used are not adequately descriptive. To address this, you request the feature engineers to extract additional features for the compounds. They have generated a new dataset, named `cruzi_comp_activity_structural_features.csv`, which includes the chemical substructures present in each compound that are known to be informative of biological activity.

Your next task is to assess the effectiveness of these newly extracted features. It is crucial to determine if they improve the model's performance. You can experiment with both sets of features or even the combination of both. With these many algorithms and parameters to set, the validation set will be very important! Report the performance of each model on a held-out test set with relevant evaluation metrics. 

In [19]:
# load the dataset
struct_df = pd.read_csv('cruzi_comp_activity_structural_features.csv')
struct_df

,CID,0,1,2,3,4,5,6,7,8,...,119,120,121,122,123,124,125,126,127,EC50
0,1201,1,0,1,1,1,0,1,0,0,...,1,1,1,1,0,0,1,1,0,2.280
1,3793,1,0,1,1,1,1,1,1,1,...,0,1,1,0,1,1,0,1,0,0.100
2,7573,1,0,0,0,1,1,1,0,0,...,1,0,0,0,0,1,0,0,0,2.524
3,17520,0,0,1,0,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0.524
4,31316,1,0,1,1,1,1,0,0,1,...,1,1,0,0,0,1,0,0,0,2.112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,56642840,1,1,1,1,1,0,1,0,1,...,1,0,0,0,1,0,1,0,1,2.415
3843,56642874,1,1,1,1,1,0,1,0,1,...,0,1,0,1,1,0,0,1,0,0.663
3844,56642875,1,0,1,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0.141
3845,56642931,1,1,1,0,1,0,1,0,1,...,0,1,0,1,0,0,0,0,0,0.854


In [ ]:
# Carregar o dataset original
mol_df = pd.read_csv('cruzi_comp_activity_molecular_features.csv')

# Verificar valores ausentes e estatísticas básicas
missing_values = mol_df.isnull().sum()
statistics = mol_df.describe()

# Tratar valores ausentes imputando com a média de cada coluna
mol_df['NumberOfAromaticRings'].fillna(mol_df['NumberOfAromaticRings'].mean(), inplace=True)
mol_df['TPSA'].fillna(mol_df['TPSA'].mean(), inplace=True)

# Criar uma variável alvo binária
mol_df['Active'] = mol_df['EC50'] <= 1

# Carregar o novo conjunto de dados de estruturas
struct_df = pd.read_csv('cruzi_comp_activity_structural_features.csv')

# Usar 'CID' como a coluna comum para mesclagem
common_column = 'CID'

# Verificar se a coluna comum está presente em ambos os datasets
if common_column in mol_df.columns and common_column in struct_df.columns:
    # Combinar os dois conjuntos de dados baseando-se no identificador comum
    combined_df = pd.merge(mol_df, struct_df, on=common_column)
else:
    raise KeyError(f"A coluna '{common_column}' não está presente em um dos datasets.")

# Verificar valores ausentes
print(combined_df.isnull().sum())

# Tratar valores ausentes, se necessário
combined_df.fillna(combined_df.mean(), inplace=True)

# Preparar os dados para o modelo
X_combined = combined_df.drop(columns=['EC50', 'Active', common_column])
y_combined = combined_df['Active']

# Dividir os dados em treino, validação e teste
X_train_combined, X_temp_combined, y_train_combined, y_temp_combined = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42)
X_val_combined, X_test_combined, y_val_combined, y_test_combined = train_test_split(X_temp_combined, y_temp_combined, test_size=0.5, random_state=42)

# Padronizar as features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)
X_val_scaled = scaler.transform(X_val_combined)
X_test_scaled = scaler.transform(X_test_combined)

# Função para avaliar o modelo
def evaluate_model(y_true, y_pred):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred)
    }

# Função para treinar e avaliar modelos
def train_and_evaluate(models, X_train, y_train, X_val, y_val):
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        results[name] = evaluate_model(y_val, y_pred)
    return results

# Modelos a serem testados
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'k-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Neural Network': MLPClassifier(max_iter=1000)
}

In [ ]:
# Avaliar modelos usando os recursos antigos
X_train_old = mol_df.drop(columns=['EC50', 'Active', 'CID'])
y_train_old = mol_df['Active']
X_train_old_scaled = scaler.fit_transform(X_train_old)
results_old_features = train_and_evaluate(models, X_train_old_scaled, y_train_old, X_val_scaled, y_val_combined)

# Avaliar modelos usando os novos recursos
X_train_new = struct_df.drop(columns=['EC50', 'Active', 'CID'])
y_train_new = struct_df['Active']
X_train_new_scaled = scaler.fit_transform(X_train_new)
results_new_features = train_and_evaluate(models, X_train_new_scaled, y_train_combined, X_val_scaled, y_val_combined)

# Avaliar modelos usando a combinação de ambos os conjuntos de recursos
results_combined_features = train_and_evaluate(models, X_train_scaled, y_train_combined, X_val_scaled, y_val_combined)

# Organizar os resultados em um DataFrame
results_df = pd.DataFrame({
    'Method': ['Logistic Regression', 'k-Nearest Neighbors', 'Decision Tree', 'Neural Network'],
    'Old Features': [results_old_features[model]['accuracy'] for model in models.keys()],
    'New Features': [results_new_features[model]['accuracy'] for model in models.keys()],
    'Combined Features': [results_combined_features[model]['accuracy'] for model in models.keys()]
})

print(results_df)

Below is a markdown example of how you can report the results! Feel free to modify it to your liking!

| **Method**        | **Metric 1** | **Metric 2** | **Metric 3** | **...** |
|-------------------|--------------|--------------|--------------|---------|
| **Log Reg**       |              |              |              |         |
| **KNN**           |              |              |              |         |
| **Neural Nets**   |              |              |              |         |
| **Random Forest** |              |              |              |         |
| **...**           |              |              |              |         |